<a href="https://colab.research.google.com/github/miniar22-mimi/ETPF_miniar_hammami/blob/main/Copie_de_Projet_Chatbot_V0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation des bibliothèques nécessaires
!pip install langchain langchain-community pypdf faiss-cpu -q
# or faiss-gpu if you have CUDA
!pip install langchain_groq -q
!pip install -U langchain-huggingface -q
!pip install gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [ ]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import HumanMessage, AIMessage
import gradio as gr
import os
import sys
import io

# Configuration de l'API Groq
os.environ["GROQ_API_KEY"] = "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"

# Fonction pour créer une nouvelle base vectorielle
def create_new_vectorstore(pdf_path, embedding_function, persist_directory):
    try:
        # Chargement du PDF
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()

        # Découpage des documents en chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=200,
            chunk_overlap=50,
            separators=["\n\n", "\n", " ", ""]
        )
        docs = text_splitter.split_documents(pages)

        # Création de la base vectorielle
        vectorstore = FAISS.from_documents(docs, embedding_function)

        # Sauvegarde de la base
        if persist_directory:
            try:
                vectorstore.save_local(persist_directory)
                print(f"Base vectorielle sauvegardée dans {persist_directory}")
            except Exception as e:
                print(f"Erreur lors de la sauvegarde de la base FAISS: {str(e)}")

        return vectorstore
    except Exception as e:
        print(f"Erreur lors de la création de la base vectorielle: {str(e)}")
        # Créer une base vectorielle avec des données minimales en cas d'échec
        return FAISS.from_texts(
            ["Tendances mode hommes: vêtements oversize, couleurs neutres.",
             "Tendances mode femmes: années 90, coupes fluides, couleurs vives."],
            embedding_function
        )

# Fonction pour charger ou créer la base vectorielle FAISS
def setup_vectorstore():
    # Chemin vers votre PDF
    pdf_path = "Cosmetiques_2025.pdf"
    persist_directory = "./faiss_db"

    # Configuration des embeddings HuggingFace
    embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Vérifier si la base FAISS existe déjà
    if os.path.exists(persist_directory):
        print("Chargement de la base FAISS existante...")
        try:
            vectorstore = FAISS.load_local(persist_directory, embedding_function, allow_dangerous_deserialization=True)
            print("Base FAISS chargée avec succès")
        except Exception as e:
            print(f"Erreur lors du chargement de la base FAISS: {str(e)}")
            print("Création d'une nouvelle base FAISS...")
            vectorstore = create_new_vectorstore(pdf_path, embedding_function, persist_directory)
    else:
        print("Création d'une nouvelle base FAISS...")
        vectorstore = create_new_vectorstore(pdf_path, embedding_function, persist_directory)

    print(f"Nombre de documents indexés : {vectorstore.index.ntotal}")
    return vectorstore

# Initialisation du modèle Groq
groq_llm = ChatGroq(model_name="llama-3.3-70b-versatile")

# Chargement ou création de la base vectorielle
try:
    vectorstore = setup_vectorstore()
except Exception as e:
    print(f"Erreur critique lors de la configuration de la base vectorielle: {str(e)}")
    # Créer une base vectorielle vide ou avec des données minimales en cas d'échec
    embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(
        ["Crème hydratante jour/nuit adaptée aux peaux sèches.",
         "Sérum à la vitamine C pour un teint éclatant."],
        embedding_function
    )

# Fonction pour sauvegarder l'historique des conversations
def save_conversation_history(history):
    try:
        with open("conversation_history.txt", "w", encoding="utf-8") as f:
            for msg in history:
                role = "Human" if isinstance(msg, HumanMessage) else "AI"
                f.write(f"{role}: {msg.content}\n")
        print("Historique de conversation sauvegardé")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde de l'historique: {str(e)}")

# Fonction pour charger l'historique des conversations
def load_conversation_history():
    try:
        if os.path.exists("conversation_history.txt"):
            history = []
            with open("conversation_history.txt", "r", encoding="utf-8") as f:
                lines = f.readlines()
                for line in lines:
                    if line.startswith("Human: "):
                        history.append(HumanMessage(content=line[7:].strip()))
                    elif line.startswith("AI: "):
                        history.append(AIMessage(content=line[4:].strip()))
            return history
        return []
    except Exception as e:
        print(f"Erreur lors du chargement de l'historique: {str(e)}")
        return []

# Initialisation de la mémoire de conversation améliorée
# Utilisation de ConversationBufferWindowMemory pour garder un nombre défini de messages
memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="question",
    output_key="answer",
    k=10  # Nombre de tours de conversation à conserver
)

# Chargement de l'historique précédent s'il existe
previous_history = load_conversation_history()
if previous_history:
    for msg in previous_history:
        if isinstance(msg, HumanMessage):
            memory.chat_memory.add_user_message(msg.content)
        elif isinstance(msg, AIMessage):
            memory.chat_memory.add_ai_message(msg.content)
    print(f"Historique chargé: {len(previous_history)} messages")

# Définition du template de prompt personnalisé avec référence explicite à l'historique
qa_prompt = PromptTemplate(
    template=(
        "Vous êtes un expert en soins et produits cosmétiques. "
        "Utilisez les informations du contexte ci-dessous pour répondre avec précision et pertinence. "
        "Adaptez vos conseils selon le type de peau, les préférences naturelles ou les besoins en maquillage.\n\n"
        "Contexte: {context}\n"
        "Question du client: {question}\n\n"
        "Réponse en tant que conseiller beauté:"
    ),
    input_variables=["context", "question"]
)

# Création de la chaîne de conversation avec récupération
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=groq_llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": qa_prompt},
    verbose=True  # Activer le mode verbeux pour le débogage
)

# Questions prédéfinies pour l'interface
sample_questions = [
    "Quels soins recommandez-vous pour peau sèche ?",
    "Quels sont les meilleurs produits de maquillage bio ?",
    "Quel sérum utiliser pour les taches pigmentaires ?",
    "Comment choisir une crème de nuit adaptée ?",
    "Quels produits capillaires pour cheveux abîmés ?"
]

# Fonction de réponse du chatbot
def chatbot(question):
    if not question.strip():
        return "Veuillez entrer une question."

    try:
        # Appel à la chaîne de conversation
        response = qa_chain({"question": question})
        answer = response['answer']

        # Sauvegarde de l'historique après chaque échange
        save_conversation_history(memory.chat_memory.messages)

        # Extraction des sources
        sources = ""
        if 'source_documents' in response and response['source_documents']:
            sources_list = []
            for i, doc in enumerate(response['source_documents']):
                try:
                    page_info = f"(Page {doc.metadata.get('page', 'N/A')})"
                    source_text = f"**Document {i+1}:** {doc.page_content} {page_info}"
                    sources_list.append(source_text)
                except Exception as e:
                    sources_list.append(f"**Document {i+1}:** [Erreur d'affichage: {str(e)}]")

            sources = "\n\n".join(sources_list)

        # Formatage de la réponse avec Markdown
        return f"**Réponse :** {answer}\n\n**Sources :**\n{sources}"

    except Exception as e:
        return f"Une erreur s'est produite: {str(e)}"

# Fonction pour afficher l'historique de conversation
def show_history():
    if hasattr(memory, 'chat_memory') and memory.chat_memory.messages:
        try:
            history_items = []
            for msg in memory.chat_memory.messages:
                role = "Vous" if isinstance(msg, HumanMessage) else "Assistant"
                content = msg.content if msg.content else "[Contenu vide]"
                history_items.append(f"**{role}:** {content}")

            history = "\n\n".join(history_items)
            return history
        except Exception as e:
            return f"Erreur lors de l'affichage de l'historique: {str(e)}"
    return "Aucun historique disponible."

# Fonction pour effacer l'historique
def clear_history():
    try:
        memory.clear()
        # Supprimer également le fichier d'historique
        if os.path.exists("conversation_history.txt"):
            os.remove("conversation_history.txt")
        return "Historique effacé."
    except Exception as e:
        return f"Erreur lors de l'effacement de l'historique: {str(e)}"

# Interface Gradio
with gr.Blocks(theme=gr.themes.Soft()) as interface:
    gr.Markdown("## Assistant Mode - LangChain & Groq\nPosez votre question sur les tendances mode 2025.")

    with gr.Row():
        question_input = gr.Textbox(placeholder="Posez votre question ici...", label="Votre question")
        submit_btn = gr.Button("Envoyer", variant="primary")

    output = gr.Markdown(label="Réponse")

    # Affichage de l'historique
    with gr.Accordion("Historique de conversation", open=False):
        history_output = gr.Markdown()

        with gr.Row():
            show_history_btn = gr.Button("Afficher l'historique")
            clear_history_btn = gr.Button("Effacer l'historique")

        show_history_btn.click(fn=show_history, outputs=history_output)
        clear_history_btn.click(fn=clear_history, outputs=history_output)

    # Boutons pour les questions prédéfinies
    gr.Markdown("### Questions Prédéfinies")
    with gr.Row():
        for i in range(0, len(sample_questions), 2):
            with gr.Column():
                for j in range(i, min(i+2, len(sample_questions))):
                    gr.Button(sample_questions[j]).click(
                        fn=chatbot,
                        inputs=gr.Textbox(value=sample_questions[j], visible=False),
                        outputs=output
                    )

    # Lien entre l'entrée manuelle et le chatbot
    submit_btn.click(chatbot, inputs=question_input, outputs=output)
    question_input.submit(chatbot, inputs=question_input, outputs=output)

# Lancement de l'interface
interface.launch(share=True)


Création d'une nouvelle base FAISS...
Erreur lors de la création de la base vectorielle: File path Cosmetiques_2025.pdf is not a valid file or url
Nombre de documents indexés : 2
Historique chargé: 1 messages
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ce3185b2a0de366a2e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
